In [129]:
import fitz  # PyMuPDF
import os
from dotenv import load_dotenv
from apify_client import ApifyClient

load_dotenv()

True

In [130]:
HUGGING_FACE_API_TOKEN = os.getenv('HUGGING_FACE_API_TOKEN')
GROQ_API_TOKEN = os.getenv('GROQ_API_TOKEN')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
# print(HUGGING_FACE_API_TOKEN)

In [131]:
# Load LLM Model
# from langchain_huggingface import HuggingFaceEndpoint
# from langchain_groq import ChatGroq
from langchain_google_genai import GoogleGenerativeAI
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

question = "Who won the FIFA World Cup in the year 2010? "

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

gemini_llm = GoogleGenerativeAI(
    model="models/gemini-2.0-flash",
    temperature=0.5,
    task='text_generation',
    max_length=500)

# repo_id = "meta-llama/Llama-3.3-70B-Instruct"

# llm = ChatGroq(
#     model="llama-3.1-8b-instant",
#     temperature=0.0,
#     max_retries=2,
#     api_key=GROQ_API_TOKEN
# )

# llama_llm = HuggingFaceEndpoint(
#     repo_id=repo_id,
#     max_length=500,
#     temperature=0.5,
#     huggingfacehub_api_token=HUGGING_FACE_API_TOKEN,
#     task='text-generation'
# )
llm_chain = LLMChain(llm=gemini_llm, prompt=prompt)
print(llm_chain.invoke({"question": question}))

Unexpected argument 'task' provided to GoogleGenerativeAI. Did you mean: 'tags'?
Unexpected argument 'max_length' provided to GoogleGenerativeAI. Did you mean: 'max_tokens'?


{'question': 'Who won the FIFA World Cup in the year 2010? ', 'text': "Okay, I'm thinking step by step... The 2010 FIFA World Cup was held in South Africa. I remember the final was between Spain and the Netherlands. And... Spain won! \n\nTherefore, the answer is **Spain**."}


In [132]:
def llm_define(llm):
    return llm

lama = llm_define(gemini_llm)

llm_chain = LLMChain(llm=lama, prompt=prompt)
print(llm_chain.invoke({"question": question}))

{'question': 'Who won the FIFA World Cup in the year 2010? ', 'text': "Okay, let's think step by step. The 2010 FIFA World Cup was held in South Africa. I remember that Spain played very well in that tournament. \n\nTherefore, the answer is **Spain**."}


In [133]:
# Load Apify api
APIFY_API_TOKEN = os.getenv('APIFY_API_TOKEN')
apify_client = ApifyClient(token=APIFY_API_TOKEN)


In [134]:
# Extract text from PDF
def extract_text_from_pdf(uploaded_file):
    doc = fitz.open(stream=uploaded_file.read(), filetype='pdf')
    text = ''
    for page in doc:
        text += page.get_text()
    return text

In [135]:
# Extract text from PDF
def extract_text_from_pdf(uploaded_file):
    doc = fitz.open(uploaded_file, filetype='pdf')
    text = ''
    for page in doc:
        text += page.get_text()
    return text

In [136]:
# Ask LLAMA to generate output
def ask_gemini(prompt, resume_text):
    llm_chain = LLMChain(llm=gemini_llm, prompt=prompt)
    response = llm_chain.invoke(resume_text)
    # response = llama_client.generate_completion(prompt=prompt, temperature=0.5)
    if isinstance(response, dict) and 'choices' in response:
    # if isinstance(response) and 'choices' in response:
        return response['choices'][0]['message']['content']
    return response

In [137]:
# Fetch Linkedin jobs using Apify
def fetch_linkedin_jobs(search_query, location="Sri Lanka", rows=60):
    run_input = {
        'title': search_query,
        'location': location,
        'rows': rows,
        'proxy': {
            'useApifyProxy': True,
            'apifyProxyGroups': ['RESIDENTIAL']
        }
    }
    run = apify_client.actor('api_key_for_apify').call(run_input=run_input)
    jobs = list(apify_client.dataset(run['defaultDatasetId']).iterate_items())
    return jobs

In [138]:
# Fetch Naukri jobs using Apify
def fetch_naukri_jobs(search_query, location="Sri Lanka", rows=60):
    run_input = {
        'keyword': search_query,
        'maxJobs': 60,
        'freshness': 'all',
        'sortBy': 'relevance',
        'experience': 'all'
    }
    run = apify_client.actor('api_key_for_apify').call(run_input=run_input)
    jobs = list(apify_client.dataset(run['defaultDatasetId']).iterate_items())
    return jobs

#### Testing

In [139]:
### Testing
# Use raw string to avoid escaping backslashes
file_path = r"C:\Users\GD\Desktop\Resume\Dilshan Upasena.pdf"

# Open the PDF
doc = fitz.open(file_path)

# Example: print the first page text
# print(doc[0].get_text())
resume_text = doc[0].get_text()
# print(resume_text)
doc[0].get_text()

'+94 72 330 8678\nAnuradhapura, Sri Lanka\ng.dilshanupasena@gmail.com\nwww.linkedin.com/in/dilshan-upasena-98gdu\nDILSHAN UPASENA\nPROFESSIONAL SUMMARY\nComputer Science undergraduate with a strong foundation in Machine Learning, Deep Learning, and Data Science. Skilled\nin developing AI-powered solutions for real-world challenges, with hands-on experience in computer vision, supervised and\nunsupervised learning. Familiar with Python and PyTorch, with practical experience in cloud deployment (AWS, Docker),\nAPI development using Flask and FastAPI, and database systems including MySQL, MongoDB, and PostgreSQL. Actively\nexploring Generative AI, including Retrieval-Augmented Generation (RAG), LangChain, and vector databases such as\nPinecone and ChromaDB. Familiar with model deployment and experimentation using platforms like Hugging Face and\nOllama, and working with large language models such as LLaMA and Gemini. Deeply passionate about academic research\nand innovative development in

In [140]:
resume_text = extract_text_from_pdf(file_path)

In [141]:
prompt_summary = PromptTemplate(
    input_variables=["resume_text"],  # this is the key you will use later
    template="Summarize this resume highlighting skills, education, and experience:\n\n{resume_text}"
)

In [142]:
summary = ask_gemini(prompt=prompt_summary, resume_text=resume_text)

In [143]:
print(summary)

{'resume_text': '+94 72 330 8678\nAnuradhapura, Sri Lanka\ng.dilshanupasena@gmail.com\nwww.linkedin.com/in/dilshan-upasena-98gdu\nDILSHAN UPASENA\nPROFESSIONAL SUMMARY\nComputer Science undergraduate with a strong foundation in Machine Learning, Deep Learning, and Data Science. Skilled\nin developing AI-powered solutions for real-world challenges, with hands-on experience in computer vision, supervised and\nunsupervised learning. Familiar with Python and PyTorch, with practical experience in cloud deployment (AWS, Docker),\nAPI development using Flask and FastAPI, and database systems including MySQL, MongoDB, and PostgreSQL. Actively\nexploring Generative AI, including Retrieval-Augmented Generation (RAG), LangChain, and vector databases such as\nPinecone and ChromaDB. Familiar with model deployment and experimentation using platforms like Hugging Face and\nOllama, and working with large language models such as LLaMA and Gemini. Deeply passionate about academic research\nand innovativ

In [145]:
LINKEDIN_APIFY_ACTOR = os.getenv('LINKEDIN_APIFY_ACTOR')
NAUKRI_APIFY_ACTOR = os.getenv('NAUKRI_APIFY_ACTOR')

print(LINKEDIN_APIFY_ACTOR)
print(NAUKRI_APIFY_ACTOR)

umUj9ttmMtFzlhTa3
alpcnRV9YI9lYVPWk
